<a href="https://colab.research.google.com/github/lmgrj/Optical-Flow-visualisation/blob/main/optical_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Farneback optical flow**

In [ ]:
#lamgarraj Mohamed


import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

def draw_flow(img, flow, step=25):
    # print(img.shape)
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    # vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    vis = img
    cv2.polylines(vis, lines, 0, (0, 255, 0), 2)
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 2, (0, 0, 255), -1)
        # vis = cv2.arrowedLine(vis, (x1,y1), (x2,y2), (0,255,0), 2) 
    return vis

video_path = '/content/output.mp4'
cap = cv2.VideoCapture('/content/drive/My Drive/Stage/Camera05.mp4')

ret, frame1 = cap.read()
width = int(frame1.shape[1] * 0.75)
height = int(frame1.shape[0] * 0.75)
frame1 = cv2.resize(frame1, (width, height), interpolation = cv2.INTER_AREA)

out = cv2.VideoWriter('result.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))

prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
print('hsv', hsv)
hsv[...,1] = 255
print('new_hsv', hsv)

while(1):
    ret, frame2 = cap.read()

    if ret == True:
        width = int(frame2.shape[1] * 0.75)
        height = int(frame2.shape[0] * 0.75)
        frame2 = cv2.resize(frame2, (width, height), interpolation = cv2.INTER_AREA)

        next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # print("flow", flow)
        
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
        bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
        frame3 = draw_flow(frame2, flow)
        nv=bgr*frame3 #***************** here
        print("************background extraction ******************* \n \n")
        cv2_imshow(bgr)
        # out.write(bgr)
        print("************vecteur flow******************* \n \n")
        cv2_imshow( frame3)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        prvs = next
    
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

# **Lucas-Kanade Optical Flow in OpenCV**


In [ ]:
import numpy as np
import cv2 as cv
import argparse
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

cap = cv.VideoCapture('/content/drive/My Drive/Stage/Camera05.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv2_imshow( img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()